In [2]:
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Activation
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import KFold

# 1. Importation de la data

In [3]:
versus=pd.read_csv("matchfiltreavecpourcentageranking2.csv",sep=';')

In [4]:
versus

,date,home_team,home_score,away_score,away_team,tournament,winner,phase_final,home_team_pourcentage_victoire,home_team_but_mis,home_team_but_enc,away_team_pourcentage_victoire,away_team_but_mis,away_team_but_enc,Rang_home_team,Rang_away_team,Host
0,1930,Spain,1,0,Czechoslovakia,0,2,0,85.71,3.1,1.1,37.50,2.2,1.7,NaN,NaN,1
1,1930,Portugal,1,0,Czechoslovakia,0,2,0,28.57,0.7,2.3,37.50,2.2,1.7,NaN,NaN,1
2,1930,Greece,2,1,Yugoslavia,0,2,0,50.00,2.5,2.8,41.67,2.4,2.1,NaN,NaN,1
3,1930,Northern Ireland,7,0,Wales,0,2,0,16.67,2.3,3.3,0.00,0.8,4.0,NaN,NaN,1
4,1930,Italy,4,2,Switzerland,0,2,0,55.56,2.8,1.6,7.69,1.7,3.7,NaN,NaN,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29648,2021,Tunisia,1,0,Egypt,0,2,0,68.42,1.7,0.7,60.00,1.8,0.5,26.0,46.0,1
29649,2021,Autre pays,1,2,Algeria,0,0,0,20.00,0.8,1.8,70.00,3.0,0.8,99.0,33.0,1
29650,2021,Autre pays,0,0,Egypt,0,1,0,20.00,0.8,1.8,60.00,1.8,0.5,99.0,46.0,1
29651,2021,Tunisia,0,2,Algeria,0,0,0,68.42,1.7,0.7,70.00,3.0,0.8,26.0,33.0,1


In [5]:
victoire_home = 0
victoire_away = 0
nul = 0
for i in range (len(versus)) :
    if versus.loc[i]["home_score"] == versus.loc[i]["away_score"]:
        nul +=1
    elif versus.loc[i]["home_score"] > versus.loc[i]["away_score"] :
        victoire_home +=1
    elif versus.loc[i]["home_score"] < versus.loc[i]["away_score"] :
        victoire_away +=1
    
poucentagevict_home = (victoire_home/ len(versus))*100
pourcentagevict_away = (victoire_away/ len(versus))*100
pourcentage_nul = (nul/len(versus))*100

In [6]:
print('Pourcentage de victoire a domicile :',poucentagevict_home)

Pourcentage de victoire a domicile : 48.17725019390956


In [7]:
print("Pourcentage de victoire a l'extérieur :",pourcentagevict_away)

Pourcentage de victoire a l'extérieur : 28.496273564226215


In [8]:
print('Pourcentage de match nul',pourcentage_nul)

Pourcentage de match nul 23.32647624186423


# 2.Traitement de la data

In [9]:
versus=versus.drop('home_score',axis=1)

In [10]:
versus=versus.drop('away_score',axis=1)

In [11]:
versus=versus.drop('home_team',axis=1)

In [12]:
versus=versus.drop('away_team',axis=1)

In [13]:
versus

,date,tournament,winner,phase_final,home_team_pourcentage_victoire,home_team_but_mis,home_team_but_enc,away_team_pourcentage_victoire,away_team_but_mis,away_team_but_enc,Rang_home_team,Rang_away_team,Host
0,1930,0,2,0,85.71,3.1,1.1,37.50,2.2,1.7,NaN,NaN,1
1,1930,0,2,0,28.57,0.7,2.3,37.50,2.2,1.7,NaN,NaN,1
2,1930,0,2,0,50.00,2.5,2.8,41.67,2.4,2.1,NaN,NaN,1
3,1930,0,2,0,16.67,2.3,3.3,0.00,0.8,4.0,NaN,NaN,1
4,1930,0,2,0,55.56,2.8,1.6,7.69,1.7,3.7,NaN,NaN,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
29648,2021,0,2,0,68.42,1.7,0.7,60.00,1.8,0.5,26.0,46.0,1
29649,2021,0,0,0,20.00,0.8,1.8,70.00,3.0,0.8,99.0,33.0,1
29650,2021,0,1,0,20.00,0.8,1.8,60.00,1.8,0.5,99.0,46.0,1
29651,2021,0,0,0,68.42,1.7,0.7,70.00,3.0,0.8,26.0,33.0,1


In [14]:
versus.dropna(axis=0,inplace=True)

In [15]:
versus['Rang_home_team']=(1.)/(versus['Rang_home_team'])

In [16]:
versus['Rang_away_team']=(1.)/(versus['Rang_away_team'])

In [17]:
versus

,date,tournament,winner,phase_final,home_team_pourcentage_victoire,home_team_but_mis,home_team_but_enc,away_team_pourcentage_victoire,away_team_but_mis,away_team_but_enc,Rang_home_team,Rang_away_team,Host
12315,1992,0,2,0,37.50,1.6,0.9,40.00,1.5,1.0,0.047619,0.055556,1
12316,1992,0,2,0,20.00,0.8,1.8,46.43,1.5,0.9,0.010101,0.037037,1
12317,1992,0,1,0,37.50,1.6,0.9,40.00,1.6,0.8,0.047619,0.071429,1
12318,1992,0,2,0,40.00,1.0,0.7,37.50,1.4,0.8,0.045455,0.024390,1
12319,1992,0,0,0,36.36,1.3,1.0,61.90,2.0,0.7,0.019608,0.076923,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
29648,2021,0,2,0,68.42,1.7,0.7,60.00,1.8,0.5,0.038462,0.021739,1
29649,2021,0,0,0,20.00,0.8,1.8,70.00,3.0,0.8,0.010101,0.030303,1
29650,2021,0,1,0,20.00,0.8,1.8,60.00,1.8,0.5,0.010101,0.021739,1
29651,2021,0,0,0,68.42,1.7,0.7,70.00,3.0,0.8,0.038462,0.030303,1


# 3.Convertion du dataframe en tableau

In [18]:
#Met y sous forme d'un tableau
y=versus[['winner']].values


In [19]:
X=versus[['Rang_home_team','Rang_away_team','Host']].values
X.shape

(16600, 3)

# 4.Séparation de la data d'entraînement de la data de test

In [20]:
#X_train,X_test,Y_train,Y_test=train_test_split(X_scaled,y_scaled,test_size=0.3,random_state=0)  #pour utiliser la version normalisé
#X_train,X_test,Y_train,y_test=train_test_split(X,y,test_size=0.3,random_state=0)

# 4.Cross validation

In [21]:
for train_index,test_index in KFold(3,shuffle=True, random_state=0).split(X):
    X_train,X_test=X[train_index],X[test_index]
    y_train,y_test=y[train_index],y[test_index]

# 5.Création d'un réseau de Neurone Artificiel

In [22]:
model = Sequential()

model.add(Dense(32, input_dim=3))
model.add(Activation('relu'))


model.add(Dense(3))
model.add(Activation('softmax'))
model.compile(loss='sparse_categorical_crossentropy',optimizer='Adam',metrics=['sparse_categorical_accuracy'])

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 32)                128       
                                                                 
 activation (Activation)     (None, 32)                0         
                                                                 
 dense_1 (Dense)             (None, 3)                 99        
                                                                 
 activation_1 (Activation)   (None, 3)                 0         
                                                                 
Total params: 227
Trainable params: 227
Non-trainable params: 0
_________________________________________________________________


# 6.entraînement du model

In [22]:
model.fit(X_train,y_train,epochs=40,batch_size=1,verbose=1)

Epoch 1/40
11067/11067 [==============================] - 13s 1ms/step - loss: 1.0125 - sparse_categorical_accuracy: 0.5080
Epoch 2/40
11067/11067 [==============================] - 13s 1ms/step - loss: 0.9779 - sparse_categorical_accuracy: 0.5391
Epoch 3/40
11067/11067 [==============================] - 13s 1ms/step - loss: 0.9572 - sparse_categorical_accuracy: 0.5534
Epoch 4/40
11067/11067 [==============================] - 13s 1ms/step - loss: 0.9446 - sparse_categorical_accuracy: 0.5642
Epoch 5/40
11067/11067 [==============================] - 12s 1ms/step - loss: 0.9372 - sparse_categorical_accuracy: 0.5682
Epoch 6/40
11067/11067 [==============================] - 13s 1ms/step - loss: 0.9346 - sparse_categorical_accuracy: 0.5665
Epoch 7/40
11067/11067 [==============================] - 13s 1ms/step - loss: 0.9315 - sparse_categorical_accuracy: 0.5714
Epoch 8/40
11067/11067 [==============================] - 13s 1ms/step - loss: 0.9290 - sparse_categorical_accuracy: 0.5750
Epoch 9/

# 7.Evaluation du model sur la data de test

In [23]:
test_loss, test_accuracy= model.evaluate(X_test,y_test)

173/173 [==============================] - 0s 1ms/step - loss: 0.9342 - sparse_categorical_accuracy: 0.5614


# 8.Prédiction / Chargement de la data à prédire

In [24]:
match_poule=pd.read_csv("matchpoulepourcentageranking2.csv",sep=';')

In [25]:
poule=match_poule

In [26]:
poule=poule.drop('winner',axis=1)
poule_Pays=match_poule


In [27]:
poule=poule.drop('home_team',axis=1)

In [28]:
poule=poule.drop('away_team',axis=1)

In [29]:
poule['Rang_home_team']=(1.)/(poule['Rang_home_team'])

In [30]:
poule['Rang_away_team']=(1.)/(poule['Rang_away_team'])

In [31]:
X_pred=poule[['Rang_home_team','Rang_away_team','Host']].values


# Affichage de la prédiction des matchs de poule

In [32]:
prediction=model.predict(X_pred)
i=0

for pred in prediction:
    print('Pour le match entre',poule_Pays['home_team'][i],'et',poule_Pays['away_team'][i])
    if (np.argmax(pred))==2 :

        print('Victoire pour',poule_Pays['home_team'][i],'\n')    


    elif (np.argmax(pred))==0:
        print('Victoire pour',poule_Pays['away_team'][i],'\n')

    else :
        print('Match null\n')


    i=i+1


2/2 [==============================] - 0s 2ms/step
Pour le match entre Qatar et Ecuador
Victoire pour Ecuador 

Pour le match entre England et Iran
Victoire pour England 

Pour le match entre Senegal et Netherlands
Victoire pour Netherlands 

Pour le match entre United States et Wales
Victoire pour United States 

Pour le match entre Argentina et Saudi Arabia
Victoire pour Argentina 

Pour le match entre Denmark et Tunisia
Victoire pour Denmark 

Pour le match entre Mexico et Poland
Victoire pour Mexico 

Pour le match entre France et Australia
Victoire pour France 

Pour le match entre Morocco et Croatia
Victoire pour Croatia 

Pour le match entre Germany et Japan
Victoire pour Germany 

Pour le match entre Spain et Costa Rica
Victoire pour Spain 

Pour le match entre Belgium et Canada
Victoire pour Belgium 

Pour le match entre Switzerland et Cameroon
Victoire pour Switzerland 

Pour le match entre Uruguay et South Korea
Victoire pour Uruguay 

Pour le match entre Portugal et Ghana
V

# Génération du fichier CSV de la sortie de notre prédiction

In [33]:
prediction=model.predict(X_pred)
i=0
groupe='0'
with open('previsionresultatsmatchdepoule.csv','w') as file:
    file.write('Pays1,Pays2,Groupe,0,1,2,Prévision_de_victoire_pour,point_pays1,points_pays2\n')
    for pred in prediction:
        #print('Pour le match entre',poule_Pays['home_team'][i],'et',poule_Pays['away_team'][i])
        if (np.argmax(pred))==2 :
            
            #print('Victoire pour',poule_Pays['home_team'][i],'\n')
            if ((poule_Pays['home_team'][i])=='Qatar' ) | ((poule_Pays['home_team'][i])=='Netherlands') | ((poule_Pays['home_team'][i])=='Senegal') | ((poule_Pays['home_team'][i])=='Ecuador'):
                groupe='A'                               
            elif ((poule_Pays['home_team'][i])=='England' ) | ((poule_Pays['home_team'][i])=='United States') | ((poule_Pays['home_team'][i])=='Iran') | ((poule_Pays['home_team'][i])=='Wales'):
                groupe='B'                
            elif ((poule_Pays['home_team'][i])=='Argentina' ) | ((poule_Pays['home_team'][i])=='Mexico') | ((poule_Pays['home_team'][i])=='Poland') | ((poule_Pays['home_team'][i])=='Saudi Arabia'):
                groupe='C'                
            elif ((poule_Pays['home_team'][i])=='France' ) | ((poule_Pays['home_team'][i])=='Denmark') | ((poule_Pays['home_team'][i])=='Tunisia') | ((poule_Pays['home_team'][i])=='Australia'):
                groupe='D'                
            elif ((poule_Pays['home_team'][i])=='Spain' ) | ((poule_Pays['home_team'][i])=='Germany') | ((poule_Pays['home_team'][i])=='Japan') | ((poule_Pays['home_team'][i])=='Costa Rica'):
                groupe='E'                
            elif ((poule_Pays['home_team'][i])=='Belgium' ) | ((poule_Pays['home_team'][i])=='Croatia') | ((poule_Pays['home_team'][i])=='Morocco') | ((poule_Pays['home_team'][i])=='Canada'):
                groupe='F'                
            elif ((poule_Pays['home_team'][i])=='Brazil' ) | ((poule_Pays['home_team'][i])=='Switzerland') | ((poule_Pays['home_team'][i])=='Serbia') | ((poule_Pays['home_team'][i])=='Cameroon'):
                groupe='G'                
            elif ((poule_Pays['home_team'][i])=='Portugal' ) | ((poule_Pays['home_team'][i])=='Uruguay') | ((poule_Pays['home_team'][i])=='South Korea') | ((poule_Pays['home_team'][i])=='Ghana'):
                groupe='H'
                        
            
            file.write(poule_Pays['home_team'][i]+','+poule_Pays['away_team'][i]+','+groupe+','+str(pred[0])+','+str(pred[1])+','+str(pred[2])+','+poule_Pays['home_team'][i]+',3,0\n')
                
        
        elif (np.argmax(pred))==0:
            #print('Victoire pour',poule_Pays['away_team'][i],'\n')
            
            if ((poule_Pays['home_team'][i])=='Qatar' ) | ((poule_Pays['home_team'][i])=='Netherlands') | ((poule_Pays['home_team'][i])=='Senegal') | ((poule_Pays['home_team'][i])=='Ecuador'):
                groupe='A'                                
            elif ((poule_Pays['home_team'][i])=='England' ) | ((poule_Pays['home_team'][i])=='United States') | ((poule_Pays['home_team'][i])=='Iran') | ((poule_Pays['home_team'][i])=='Wales'):
                groupe='B'                
            elif ((poule_Pays['home_team'][i])=='Argentina' ) | ((poule_Pays['home_team'][i])=='Mexico') | ((poule_Pays['home_team'][i])=='Poland') | ((poule_Pays['home_team'][i])=='Saudi Arabia'):
                groupe='C'                
            elif ((poule_Pays['home_team'][i])=='France' ) | ((poule_Pays['home_team'][i])=='Denmark') | ((poule_Pays['home_team'][i])=='Tunisia') | ((poule_Pays['home_team'][i])=='Australia'):
                groupe='D'                
            elif ((poule_Pays['home_team'][i])=='Spain' ) | ((poule_Pays['home_team'][i])=='Germany') | ((poule_Pays['home_team'][i])=='Japan') | ((poule_Pays['home_team'][i])=='Costa Rica'):
                groupe='E'                
            elif ((poule_Pays['home_team'][i])=='Belgium' ) | ((poule_Pays['home_team'][i])=='Croatia') | ((poule_Pays['home_team'][i])=='Morocco') | ((poule_Pays['home_team'][i])=='Canada'):
                groupe='F'                
            elif ((poule_Pays['home_team'][i])=='Brazil' ) | ((poule_Pays['home_team'][i])=='Switzerland') | ((poule_Pays['home_team'][i])=='Serbia') | ((poule_Pays['home_team'][i])=='Cameroon'):
                groupe='G'                
            elif ((poule_Pays['home_team'][i])=='Portugal' ) | ((poule_Pays['home_team'][i])=='Uruguay') | ((poule_Pays['home_team'][i])=='South Korea') | ((poule_Pays['home_team'][i])=='Ghana'):
                groupe='H'
                
                
            file.write(poule_Pays['home_team'][i]+','+poule_Pays['away_team'][i]+','+groupe+','+str(pred[0])+','+str(pred[1])+','+str(pred[2])+','+poule_Pays['away_team'][i]+',0,3\n')
            
        else :
            #print('Match null\n')
            if ((poule_Pays['home_team'][i])=='Qatar' ) | ((poule_Pays['home_team'][i])=='Netherlands') | ((poule_Pays['home_team'][i])=='Senegal') | ((poule_Pays['home_team'][i])=='Ecuador'):
                groupe='A'                               
            elif ((poule_Pays['home_team'][i])=='England' ) | ((poule_Pays['home_team'][i])=='United States') | ((poule_Pays['home_team'][i])=='Iran') | ((poule_Pays['home_team'][i])=='Wales'):
                groupe='B'                
            elif ((poule_Pays['home_team'][i])=='Argentina' ) | ((poule_Pays['home_team'][i])=='Mexico') | ((poule_Pays['home_team'][i])=='Poland') | ((poule_Pays['home_team'][i])=='Saudi Arabia'):
                groupe='C'                
            elif ((poule_Pays['home_team'][i])=='France' ) | ((poule_Pays['home_team'][i])=='Denmark') | ((poule_Pays['home_team'][i])=='Tunisia') | ((poule_Pays['home_team'][i])=='Australia'):
                groupe='D'                
            elif ((poule_Pays['home_team'][i])=='Spain' ) | ((poule_Pays['home_team'][i])=='Germany') | ((poule_Pays['home_team'][i])=='Japan') | ((poule_Pays['home_team'][i])=='Costa Rica'):
                groupe='E'                
            elif ((poule_Pays['home_team'][i])=='Belgium' ) | ((poule_Pays['home_team'][i])=='Croatia') | ((poule_Pays['home_team'][i])=='Morocco') | ((poule_Pays['home_team'][i])=='Canada'):
                groupe='F'                
            elif ((poule_Pays['home_team'][i])=='Brazil' ) | ((poule_Pays['home_team'][i])=='Switzerland') | ((poule_Pays['home_team'][i])=='Serbia') | ((poule_Pays['home_team'][i])=='Cameroon'):
                groupe='G'                
            elif ((poule_Pays['home_team'][i])=='Portugal' ) | ((poule_Pays['home_team'][i])=='Uruguay') | ((poule_Pays['home_team'][i])=='South Korea') | ((poule_Pays['home_team'][i])=='Ghana'):
                groupe='H'
                
                
            file.write(poule_Pays['home_team'][i]+','+poule_Pays['away_team'][i]+','+groupe+','+str(pred[0])+','+str(pred[1])+','+str(pred[2])+','+'Match null'+',1,1\n')

        i=i+1
    print('fichier généré')

2/2 [==============================] - 0s 2ms/step
fichier généré


# Génération du Tableau de point de chaque poule

In [34]:
prediction=model.predict(X_pred)
i=0
groupe='0'

listepays={'Qatar':[0,'A',0,0],'Brazil':[0,'G',0,0],'Belgium':[0,'F',0,0],'France':[0,'D',0,0],'Argentina':[0,'C',0,0],'England':[0,'B',0,0],
           'Spain':[0,'E',0,0],'Portugal':[0,'H',0,0],'Mexico':[0,'C',0,0],'Netherlands':[0,'A',0,0],'Denmark':[0,'D',0,0],'Germany':[0,'E',0,0],
           'Uruguay':[0,'H',0,0],'Switzerland':[0,'G',0,0],'United States':[0,'B',0,0],'Croatia':[0,'F',0,0],'Senegal':[0,'A',0,0],
           'Iran':[0,'B',0,0],'Japan':[0,'E',0,0],'Morocco':[0,'F',0,0],'Serbia':[0,'G',0,0],'Poland':[0,'C',0,0],'South Korea':[0,'H',0,0],
           'Tunisia':[0,'D',0,0],'Cameroon':[0,'G',0,0],'Canada':[0,'F',0,0],'Ecuador':[0,'A',0,0],'Saudi Arabia':[0,'C',0,0],'Ghana':[0,'H',0,0],
           'Wales':[0,'B',0,0],'Costa Rica':[0,'E',0,0],'Australia':[0,'D',0,0]     
          
          }



for pred in prediction:
    #print('Pour le match entre',poule_Pays['home_team'][i],'et',poule_Pays['away_team'][i])
    if (np.argmax(pred))==2 :
        listepays[poule_Pays['home_team'][i]][0]=listepays[poule_Pays['home_team'][i]][0]+3
        listepays[poule_Pays['home_team'][i]][2]=listepays[poule_Pays['home_team'][i]][2]+pred[2]
        listepays[poule_Pays['home_team'][i]][3]=poule_Pays['Rang_home_team'][i]
        #print([poule_Pays['home_team'][i]],listepays[poule_Pays['home_team'][i]])



    elif (np.argmax(pred))==0:
        #print('Victoire pour',poule_Pays['away_team'][i],'\n')
        listepays[poule_Pays['away_team'][i]][0]=listepays[poule_Pays['away_team'][i]][0]+3
        listepays[poule_Pays['away_team'][i]][2]=listepays[poule_Pays['away_team'][i]][2]+(pred[0])
        listepays[poule_Pays['away_team'][i]][3]=poule_Pays['Rang_away_team'][i]

        #print([poule_Pays['home_team'][i]],listepays[poule_Pays['away_team'][i]][0])


    else :
        listepays[poule_Pays['away_team'][i]][0]=listepays[poule_Pays['away_team'][i]][0]+1
        listepays[poule_Pays['away_team'][i]][2]=listepays[poule_Pays['away_team'][i]][2]+(pred[1]/3)
        print(listepays[poule_Pays['away_team'][i]][0])
        listepays[poule_Pays['home_team'][i]][0]=listepays[poule_Pays['home_team'][i]][0]+1
        listepays[poule_Pays['home_team'][i]][2]=listepays[poule_Pays['home_team'][i]][2]+(pred[1]/3)
        #print([poule_Pays['home_team'][i]],listepays[poule_Pays['home_team'][i]][0])


    i=i+1
#print(listepays)
        
with open('previsionpointpoule.csv','w') as file:
    file.write('Pays,Groupe,points,score,rang\n')
    for keys in listepays:
        #print(keys)
        file.write(keys+','+str(listepays[keys][1])+','+str(listepays[keys][0])+','+str(listepays[keys][2])+','+str(listepays[keys][3])+'\n')


    print('fichier généré')

2/2 [==============================] - 0s 2ms/step
fichier généré


In [35]:
#df2=pd.DataFrame(listepays.keys(),listepays.values(),columns=['Pays3','Pays','test','Paysh','testh'])


In [36]:
df2=pd.DataFrame(listepays.values(),columns=['Points','groupe','score','rang'])


In [37]:
df2['Pays']=pd.DataFrame(listepays.keys(),columns=['Pays'])

In [38]:
groupeA=df2[(df2['groupe']=='A')].sort_values(by='score',ascending=False).head(2)
groupeB=df2[(df2['groupe']=='B')].sort_values(by='score',ascending=False).head(2)
groupeC=df2[(df2['groupe']=='C')].sort_values(by='score',ascending=False).head(2)
groupeD=df2[(df2['groupe']=='D')].sort_values(by='score',ascending=False).head(2)
groupeE=df2[(df2['groupe']=='E')].sort_values(by='score',ascending=False).head(2)
groupeF=df2[(df2['groupe']=='F')].sort_values(by='score',ascending=False).head(2)
groupeG=df2[(df2['groupe']=='G')].sort_values(by='score',ascending=False).head(2)
groupeH=df2[(df2['groupe']=='H')].sort_values(by='score',ascending=False).head(2)


In [39]:
dfpoule=pd.DataFrame({'home_team':[],'Rang_home_team':[],'away_team':[],'Rang_away_team':[]})

In [40]:
df1=pd.DataFrame()
df2=pd.DataFrame()

In [41]:
df1=pd.DataFrame([[groupeA['Pays'].head(1).values[0],groupeA['rang'].head(1).values[0],groupeB['Pays'].tail(1).values[0],groupeB['rang'].tail(1).values[0]]],columns=['home_team','Rang_home_team','away_team','Rang_away_team'])

In [42]:
df2=pd.DataFrame([[groupeC['Pays'].head(1).values[0],groupeC['rang'].head(1).values[0],groupeD['Pays'].tail(1).values[0],groupeD['rang'].tail(1).values[0]]],columns=['home_team','Rang_home_team','away_team','Rang_away_team'])

In [43]:
df3=pd.DataFrame([[groupeE['Pays'].head(1).values[0],groupeE['rang'].head(1).values[0],groupeF['Pays'].tail(1).values[0],groupeF['rang'].tail(1).values[0]]],columns=['home_team','Rang_home_team','away_team','Rang_away_team'])

In [44]:
df4=pd.DataFrame([[groupeG['Pays'].head(1).values[0],groupeG['rang'].head(1).values[0],groupeH['Pays'].tail(1).values[0],groupeH['rang'].tail(1).values[0]]],columns=['home_team','Rang_home_team','away_team','Rang_away_team'])

In [45]:
df5=pd.DataFrame([[groupeB['Pays'].head(1).values[0],groupeB['rang'].head(1).values[0],groupeA['Pays'].tail(1).values[0],groupeA['rang'].tail(1).values[0]]],columns=['home_team','Rang_home_team','away_team','Rang_away_team'])

In [46]:
df6=pd.DataFrame([[groupeD['Pays'].head(1).values[0],groupeD['rang'].head(1).values[0],groupeC['Pays'].tail(1).values[0],groupeC['rang'].tail(1).values[0]]],columns=['home_team','Rang_home_team','away_team','Rang_away_team'])

In [47]:
df7=pd.DataFrame([[groupeF['Pays'].head(1).values[0],groupeF['rang'].head(1).values[0],groupeE['Pays'].tail(1).values[0],groupeE['rang'].tail(1).values[0]]],columns=['home_team','Rang_home_team','away_team','Rang_away_team'])

In [48]:
df8=pd.DataFrame([[groupeH['Pays'].head(1).values[0],groupeH['rang'].head(1).values[0],groupeG['Pays'].tail(1).values[0],groupeG['rang'].tail(1).values[0]]],columns=['home_team','Rang_home_team','away_team','Rang_away_team'])

In [49]:
dfpoule=pd.concat([dfpoule,df1], ignore_index=True)
dfpoule=pd.concat([dfpoule,df2], ignore_index=True)
dfpoule=pd.concat([dfpoule,df3], ignore_index=True)
dfpoule=pd.concat([dfpoule,df4], ignore_index=True)
dfpoule=pd.concat([dfpoule,df5], ignore_index=True)
dfpoule=pd.concat([dfpoule,df6], ignore_index=True)
dfpoule=pd.concat([dfpoule,df7], ignore_index=True)
dfpoule=pd.concat([dfpoule,df8], ignore_index=True)


In [50]:
dfpoule

,home_team,Rang_home_team,away_team,Rang_away_team
0,Netherlands,8.0,United States,16.0
1,Argentina,3.0,Denmark,10.0
2,Spain,7.0,Croatia,12.0
3,Brazil,1.0,Uruguay,14.0
4,England,5.0,Senegal,18.0
5,France,4.0,Mexico,13.0
6,Belgium,2.0,Germany,11.0
7,Portugal,9.0,Switzerland,15.0


In [51]:
dfpoule.insert(4,'Host',0)

In [52]:
match_poule=dfpoule
poule=match_poule

poule_Pays=match_poule
poule=poule.drop('home_team',axis=1)
poule=poule.drop('away_team',axis=1)
poule['Rang_home_team']=(1.)/(poule['Rang_home_team'])
poule['Rang_away_team']=(1.)/(poule['Rang_away_team'])
X_pred=poule[['Rang_home_team','Rang_away_team','Host']]

# Prédiction des matchs de 8eme

In [53]:
#match_poule=pd.read_csv("matchdes8.csv",sep=';')
#poule=match_poule
#poule=poule.drop('winner',axis=1)
#poule_Pays=match_poule
#poule=poule.drop('home_team',axis=1)
#poule=poule.drop('away_team',axis=1)
#poule['Rang_home_team']=(1.)/(poule['Rang_home_team'])
#poule['Rang_away_team']=(1.)/(poule['Rang_away_team'])
#X_pred=poule[['Rang_home_team','Rang_away_team','Host']].values

In [54]:
#X_pred

In [55]:
poule_Pays['home_team'][0]

'Netherlands'

# Affichage des resultats de 8eme

In [56]:
prediction=model.predict(X_pred)
i=0

for pred in prediction:
    print('Pour le match entre',poule_Pays['home_team'][i],'et',poule_Pays['away_team'][i])
    if (np.argmax(pred))==2 :

        print('Victoire pour',poule_Pays['home_team'][i],'\n')    


    elif (np.argmax(pred))==0:
        print('Victoire pour',poule_Pays['away_team'][i],'\n')

    else :
        print('Match null\n')


    i=i+1

1/1 [==============================] - 0s 50ms/step
Pour le match entre Netherlands et United States
Victoire pour Netherlands 

Pour le match entre Argentina et Denmark
Victoire pour Argentina 

Pour le match entre Spain et Croatia
Victoire pour Spain 

Pour le match entre Brazil et Uruguay
Victoire pour Brazil 

Pour le match entre England et Senegal
Victoire pour England 

Pour le match entre France et Mexico
Victoire pour France 

Pour le match entre Belgium et Germany
Victoire pour Belgium 

Pour le match entre Portugal et Switzerland
Victoire pour Portugal 



# Génération du fichier CSV de la sortie de notre prédiction des 8eme

In [57]:
prediction=model.predict(X_pred)
i=0
host=0
Rang=0
Pays=''



quart={'Rang_home_team':[],'Rang_away_team':[],'Host':[],'home_team':[],'away_team':[]}

with open('previsionresultats8eme.csv','w') as file:
    file.write('Pays1,Pays2,0,1,2,Prévision_de_victoire_pour\n')
    for pred in prediction:
        #print('Pour le match entre',poule_Pays['home_team'][i],'et',poule_Pays['away_team'][i])
        if (np.argmax(pred))==2 :
                          
            file.write(poule_Pays['home_team'][i]+','+poule_Pays['away_team'][i]+','+str(pred[0])+','+str(pred[1])+','+str(pred[2])+','+poule_Pays['home_team'][i]+'\n')
            Rang=poule['Rang_home_team'][i]
            Pays=poule_Pays['home_team'][i]
            if(poule_Pays['home_team'][i]=='Qatar'):
                host=1
        
        elif (np.argmax(pred))==0:
      
            file.write(poule_Pays['home_team'][i]+','+poule_Pays['away_team'][i]+','+str(pred[0])+','+str(pred[1])+','+str(pred[2])+','+poule_Pays['away_team'][i]+'\n')
            Rang=poule_Pays['Rang_away_team'][i]
            Pays=poule_Pays['away_team'][i]
        else :
                           
            file.write(poule_Pays['home_team'][i]+','+poule_Pays['away_team'][i]+','+str(pred[0])+','+str(pred[1])+','+str(pred[2])+','+'Match null'+'\n')
        
        
        
        
        if (i%2)==0:
            quart['Rang_home_team'].append(Rang)
            quart['home_team'].append(Pays)
            quart['Host'].append(host)
        else:
            quart['Rang_away_team'].append(Rang)
            quart['away_team'].append(Pays)

        host=0

        i=i+1
    print('fichier généré')

1/1 [==============================] - 0s 20ms/step
fichier généré


In [58]:
def affichage_prediction(niv):
    dfniv=pd.DataFrame(data=niv)
    X_pred2=dfniv[['Rang_home_team','Rang_away_team','Host']].values
    prediction=model.predict(X_pred2)


    i=0

    for pred in prediction:
        print('Pour le match entre',niv['home_team'][i],'et',niv['away_team'][i])
        if (np.argmax(pred))==2 :

            print('Victoire pour',niv['home_team'][i],'\n')    


        elif (np.argmax(pred))==0:
            print('Victoire pour',niv['away_team'][i],'\n')

        else :
            print('Match null\n')
        

        i=i+1
    return(prediction)

In [59]:
def generation_csv(status,titre,prediction):
    dfstatus=pd.DataFrame(data=status)
    


    i=0
    host=0
    Rang=0
    Pays=''



    status2={'Rang_home_team':[],'Rang_away_team':[],'Host':[],'home_team':[],'away_team':[]}

    with open(f'previsionresultat{titre}.csv','w') as file:
        file.write('Pays1,Pays2,0,1,2,Prévision_de_victoire_pour\n')
        for pred in prediction:

            if (np.argmax(pred))==2 :

                file.write(status['home_team'][i]+','+status['away_team'][i]+','+str(pred[0])+','+str(pred[1])+','+str(pred[2])+','+status['home_team'][i]+'\n')
                Rang=status['Rang_home_team'][i]
                Pays=status['home_team'][i]
                if(status['home_team'][i]=='Qatar'):
                    host=1

            elif (np.argmax(pred))==0:

                file.write(status['home_team'][i]+','+status['away_team'][i]+','+str(pred[0])+','+str(pred[1])+','+str(pred[2])+','+status['away_team'][i]+'\n')
                Rang=status['Rang_away_team'][i]
                Pays=status['away_team'][i]
            else :

                file.write(status['home_team'][i]+','+status['away_team'][i]+','+str(pred[0])+','+str(pred[1])+','+str(pred[2])+','+'Match null'+'\n')




            if (i%2)==0:
                status2['Rang_home_team'].append(Rang)
                status2['home_team'].append(Pays)
                status2['Host'].append(host)
            else:
                status2['Rang_away_team'].append(Rang)
                status2['away_team'].append(Pays)

            host=0

            i=i+1
        print('fichier généré')
        return (status2)


# Affichage de la prédiction des quarts de finale

In [60]:
predictionquart=affichage_prediction(quart)

1/1 [==============================] - 0s 19ms/step
Pour le match entre Netherlands et Argentina
Victoire pour Argentina 

Pour le match entre Spain et Brazil
Victoire pour Brazil 

Pour le match entre England et France
Victoire pour France 

Pour le match entre Belgium et Portugal
Victoire pour Belgium 



# Génération du fichier CSV des quarts de finale

In [61]:
demi=generation_csv(quart,'quart',predictionquart)

fichier généré


# Affichage de la prédiction des demie-finales

In [62]:
predictiondemi=affichage_prediction(demi)

1/1 [==============================] - 0s 20ms/step
Pour le match entre Argentina et Brazil
Victoire pour Brazil 

Pour le match entre France et Belgium
Victoire pour Belgium 



# Génération du fichier CSV pour les demie-finales

In [63]:
final=generation_csv(demi,'demi',predictiondemi)

fichier généré


# Affichage de la prédiction pour la finale

In [64]:
predictionfinal=affichage_prediction(final)

1/1 [==============================] - 0s 19ms/step
Pour le match entre Brazil et Belgium
Victoire pour Belgium 



# Génération du fichier CSV de la finale

In [92]:
vainqueur=generation_csv(final,'final',predictionfinal)

fichier généré


In [93]:
print('Le vainqueur de la Coupe du monde sera : ',vainqueur['home_team'][0])

Le vainqueur de la Coupe du monde sera :  Belgium
